In [1]:
import json
from BuildGraphModule import *

In [2]:
with open('paper.json', 'r') as f:
    data = json.load(f)

In [3]:
len(data)

124620

In [4]:
authors = get_authors(data, 'Authors')

In [15]:
len(set(authors))

170649

In [6]:
authors_epfl = get_authors(data, 'Epfl authors')
# Due to the format which the authors have been saved

# We initialize a new list
epfl_authors = []

# Then for each element in the previous list
for i in authors_epfl:
    # We unpack the content
    epfl_authors += list(i.keys())

In [7]:
len(set(epfl_authors))

14479

In [8]:
# All
dictionary_a_id, dictionary_id_a = create_authors_id(authors, 'all')
# EPFL
dictionary_a_id_epfl, dictionary_id_a_epfl = create_authors_id(epfl_authors, 'epfl')

In [9]:
# All
dict_authors = list_coauthors(data)
# EPFL - COAUTORI NELLA LISTA DI EPFL AUTHORS
dict_authors_epfl = list_coauthors_epfl(data)

In [10]:
# All
dict_numb_coll = number_collaborations(dict_authors)
# EPFL
dict_numb_coll_epfl = number_collaborations(dict_authors_epfl)

In [11]:
# All
dict_authors_set = set_coauthors(dict_authors)
# EPFL
dict_authors_set_epfl = set_coauthors(dict_authors_epfl)

In [12]:
# All
edges = create_edges(dict_authors_set, dictionary_a_id)
# EPFL
edges_epfl = create_edges(dict_authors_set_epfl, dictionary_a_id_epfl)

In [13]:
len(edges_epfl)

47429

In [14]:
G = create_simple_graph(dictionary_id_a_epfl, edges_epfl)

Name: 
Type: Graph
Number of nodes: 14479
Number of edges: 47429
Average degree:   6.5514
